In [2]:
from zipfile import ZipFile
from pathlib import Path

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import cupy 
import cudf

In [3]:
df_dev = pd.read_csv(Path.cwd()/'data'/'DSL2122_january_dataset'/'development.csv')
df_eval = pd.read_csv(Path.cwd()/'data'/'DSL2122_january_dataset'/'evaluation.csv')

In [4]:
df_eval.shape

(74999, 5)

In [5]:
df = df_dev.append(df_eval)

In [6]:
# df['date'] = df['date'].\
#     astype('string')\
#     .str.split(' ')\
#     .apply(lambda x : ' '.join([x[i] for i in [1,2,3,5]]))\
#     .pipe(pd.to_datetime)

### Stopwords analysis: sklearn, nltk, stop_words

#### nltk

In [7]:
from nltk import download as nltk_download

nltk_download('stopwords')

from nltk.corpus import stopwords

nltk_stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /home/edoch/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### sklearn

In [8]:
from sklearn.feature_extraction import text

sklearn_stopwords = list(text.ENGLISH_STOP_WORDS)

#### stop_words

In [9]:
# from stop_words import get_stop_words

# stop_words_stopwords = get_stop_words('english')

I should try to use both all stopwords, and sklearn and nltk stopwords singularly (and also no stopwords)

In [10]:
def stopwords_list_gen(source_list = [sklearn_stopwords,nltk_stopwords], generate_neg = True):
    stopwords_all_list = set()
    for source in source_list:
        stopwords_all_list = stopwords_all_list.union(set(source))
        if generate_neg:
            stopwords_all_list = stopwords_all_list.union(set([f'{i+"_neg"}' for i in source]))
    return stopwords_all_list

## Preprocessing

Tokenization

In [11]:
df['text_token'] = df['text'].str.split()

Removal of '@words', '&words', 'http:words'

In [12]:
df['text_noAt'] = df['text_token'].apply(lambda x : [i for i in x if not(i.startswith('@'))])\
    .apply(lambda x : [i for i in x if not(i.startswith('&'))])\
    .apply(lambda x : [i for i in x if not(i.startswith('http:'))])

Removal of '&amp', '&quot'

In [13]:
df['text_noAmpQuot'] = df['text_noAt']\
    .apply(lambda x : [i for i in x if '&amp' not in i])\
    .apply(lambda x : [i for i in x if '&quot' not in i])

Removal of repeated letters (incomplete)

In [14]:
a = 'ulaaa'

def remove_repeated_letters(word):
    for letter in word:
        print(letter)

# nltk_stemmer.stem(a)

Remove puntuation

In [15]:
import string as py_string
df['text_noPunct'] = df['text_noAmpQuot']\
    .apply(lambda x : [i.translate(str.maketrans('', '', py_string.punctuation)) for i in x])\
    .apply(lambda x : [i for i in x if i != ''])

Negation

In [16]:
from nltk.sentiment.util import mark_negation

df['text_neg'] = df['text_noPunct'].apply(lambda x : mark_negation(x))

Removal of stopwords

In [17]:
# considered_stopwords = stop_words_stopwords + [f'{word}_neg' for word in stop_words_stopwords]

# df['text_noStopwords'] = df['text_neg'].apply(lambda x : [i for i in x if i not in considered_stopwords])

Stemming and lemmatization

In [18]:
# from nltk.stem import PorterStemmer, LancasterStemmer, SnowballStemmer, WordNetLemmatizer

# nltk_download('wordnet')
# nltk_download('omw-1.4')

# nltk_stemmer = PorterStemmer()
# # nltk_stemmer = LancasterStemmer()
# # nltk_stemmer = SnowballStemmer('english')
# nltk_lemmatizer = WordNetLemmatizer()
# # from nltk.corpus import wordnet

# df['text_stemmed'] = df['text_noPunct'].apply(lambda x : [nltk_stemmer.stem(word) for word in x])
# df['text_stemmed'] = df['text_noPunct'].apply(lambda x : [nltk_lemmatizer.lemmatize(word) for word in x])

User manual filter

In [19]:
df = df.loc[
    (~(df['user'] == 'lost_dog') &
    ~(df['user'] == 'webwoke') &
    ~(df['user'] == 'tweetpet') &
    ~(df['user'].str.contains('tweeteradder')) &
    ~(df['user'].str.contains('tweetfollow')) &
    ~(df['user'] == 'divxdownloads')) |
    df['sentiment'].isna()
]

In [20]:
df[df['sentiment'].isna()].shape

(74999, 11)

## Model creation

In [21]:
from sklearn.model_selection import ParameterGrid

multinomialNB_params = {
    'alpha' : [0,.5,1,2],
    'fit_prior' : [True]
}

TfidfVectorizer_params = {
    'stop_words' : [None],
    'ngram_range' : [(1,1)],
    'max_features' : [None, 15000],
    'max_df' : [1.0,.1,.005],
    'min_df' : [1,.0001,.00001],
    'binary' : [True,False],
    'norm' : ['l1','l2'],
    'use_idf' : [True],
    'smooth_idf' : [True,False],
    'sublinear_tf' : [False]
}

In [22]:
print(len(list(ParameterGrid(TfidfVectorizer_params)))*len(list(ParameterGrid(multinomialNB_params))))

print(len(list(ParameterGrid(TfidfVectorizer_params)))*len(list(ParameterGrid(multinomialNB_params)))*13/60/60)

576
2.08


# Train test split

In [23]:
# from sklearn.model_selection import ParameterGrid
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.svm import LinearSVC
# from sklearn.metrics import f1_score, classification_report, confusion_matrix

# from datetime import datetime
# from pathlib import Path

# logs_path = Path.cwd()/'logs'
# results_path = Path.cwd()/'results'
# logs_path.mkdir(exist_ok=True)
# results_path.mkdir(exist_ok=True)
# log_file_name = logs_path/f'log_out_{datetime.now()}.txt'
# results_file_name = results_path/f'results_out_{datetime.now()}.txt'
# with open(log_file_name, 'w') as file_log:
#     file_log.write('File output\n\n')
# with open(results_file_name, 'w') as file_result:
#     file_result.write('configuration_number,vectorizer_param,model_param,f1\n')

# df_final = df
# # df_final['text_final'] = df_final['text_noPunct'].apply(lambda x : ' '.join(x))
# df_final['text_final'] = df_final['text_neg'].apply(lambda x : ' '.join(x))

# mask_train_test = df_final['sentiment'].notna()

# X_train_valid = df_final.loc[mask_train_test,:]['text_final'].values
# y_train_valid = df_final.loc[mask_train_test,:]['sentiment'].values
# X_test = df_final.loc[~mask_train_test,:]['text_final'].values

# X_train, X_valid, y_train, y_valid = train_test_split(
#     X_train_valid, 
#     y_train_valid, 
#     shuffle=True, 
#     # stratify=y_train_valid, 
#     train_size=0.8, 
#     random_state=50
# )

# configuration_number = 0
# print(f'Number of combinations:\t{len(list(ParameterGrid(TfidfVectorizer_params)))*len(list(ParameterGrid(linearSVC_params)))}')
# for vectorizer_param in ParameterGrid(TfidfVectorizer_params):
#     for model_param in ParameterGrid(linearSVC_params):
#         configuration_number += 1
#         print(configuration_number, vectorizer_param, model_param)

#         pipe = Pipeline([
#             ('tfidf', TfidfVectorizer(**vectorizer_param)),
#             ('linearSVC', LinearSVC(**model_param))
#         ])

#         pipe.fit(X_train, y_train)

#         f1 = f1_score(y_valid, pipe.predict(X_valid),average='macro')
#         report = classification_report(y_valid, pipe.predict(X_valid))
#         # confusion = confusion_matrix(y_valid, pipe.predict(X_valid))

#         with open(log_file_name, 'a') as file_log:
#             file_log.write(f'configuration_number: {configuration_number}\n')
#             file_log.write(f'vectorizer_param: \t{vectorizer_param}\n')
#             file_log.write(f'model_param: \t\t{model_param}\n')
#             file_log.write(f'f1_score:\t\t\t{f1}\n\n')
#             file_log.write(f'{report}\n\n')
#             file_log.write(f'{"*"*150}\n\n')
#         with open(results_file_name, 'a') as file_result:
#             file_result.write(f'{configuration_number},{vectorizer_param},{model_param},{f1}\n')

# Grid search

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score, classification_report, confusion_matrix

df_final = df
# df_final['text_final'] = df_final['text_noPunct'].apply(lambda x : ' '.join(x))
df_final['text_final'] = df_final['text_neg'].apply(lambda x : ' '.join(x))

mask_train_test = df_final['sentiment'].notna()

X_train_valid = df_final.loc[mask_train_test,:]['text_final'].values
y_train_valid = df_final.loc[mask_train_test,:]['sentiment'].values
X_test = df_final.loc[~mask_train_test,:]['text_final'].values

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_valid, 
    y_train_valid, 
    shuffle=True, 
    # stratify=y_train_valid, 
    train_size=0.8, 
    random_state=50
)

vectorizer = 'tfidf'
model = 'multinomialNB'

pipe = Pipeline([
    (vectorizer, TfidfVectorizer()),
    (model, MultinomialNB())
])

def params_for_GridSearchCV(params_IN,step_name):
    return {f'{step_name}__{key}':value for (key,value) in params_IN.items()}

params_all = params_for_GridSearchCV(TfidfVectorizer_params,vectorizer)
params_all.update(params_for_GridSearchCV(multinomialNB_params,model))
print(params_all)

gscv = GridSearchCV(pipe, params_all, cv = 3, verbose = 4, scoring='f1_macro')

gscv.fit(X_train, y_train)

f1 = f1_score(y_valid, gscv.predict(X_valid),average='macro')
report = classification_report(y_valid, gscv.predict(X_valid))
confusion = confusion_matrix(y_valid, gscv.predict(X_valid))

print(f1)
print(report)
print(confusion)


In [25]:
gscv.cv_results_

{'mean_fit_time': array([1.0028553 , 0.99038569, 0.97524023, 0.97708265, 0.90424355,
        0.90019178, 0.90505958, 0.90006137, 0.92375787, 0.92578952,
        0.92634829, 0.92595092, 0.96999566, 0.97746706, 0.97578899,
        0.97507119, 0.96737162, 0.97087582, 0.96286758, 0.96016542,
        0.9727335 , 0.97163232, 0.97179953, 0.97289427, 0.95651444,
        0.94839199, 0.95526926, 0.95583995, 0.8848029 , 0.8880105 ,
        0.88645871, 0.8825833 , 0.90607492, 0.90706952, 0.90586877,
        0.90771365, 0.96391582, 0.96312086, 0.95963446, 0.95928804,
        0.95537448, 0.95637226, 0.95574792, 0.94824719, 0.96070902,
        0.96306229, 0.96135203, 0.96231135, 0.9379576 , 0.94246054,
        0.93936117, 0.93904932, 0.86633054, 0.86860553, 0.86872538,
        0.87089864, 0.89024631, 0.89068516, 0.89047011, 0.89042711,
        0.94852543, 0.94774834, 0.94200261, 0.94849586, 0.94119747,
        0.93537005, 0.93882442, 0.93906649, 0.94901872, 0.94515888,
        0.95144868, 0.94803023,

In [28]:
pd.DataFrame(gscv.cv_results_).to_csv('multinomialBayes_results.csv')

In [4]:
a = pd.read_csv('multinomialBayes_results.csv')

In [7]:
a.loc[a['rank_test_score']==1]

,Unnamed: 0,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_multinomialNB__alpha,param_multinomialNB__fit_prior,param_tfidf__binary,param_tfidf__max_df,param_tfidf__max_features,...,param_tfidf__stop_words,param_tfidf__sublinear_tf,param_tfidf__use_idf,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
22,22,0.9718,0.003424,0.461028,0.000866,0.0,True,True,1.0,15000.0,...,NaN,False,True,"{'multinomialNB__alpha': 0, 'multinomialNB__fi...",0.727403,0.727689,0.727369,0.727487,0.000143,1


# cuML

In [26]:
# from sklearn.model_selection import ParameterGrid
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# from cuml.feature_extraction.text import TfidfVectorizer as cuml_TfidfVectorizer
# from sklearn.svm import LinearSVC
# from cuml.svm import LinearSVC as cuml_LinearSVC
# from sklearn.metrics import f1_score, classification_report, confusion_matrix

# from datetime import datetime
# from pathlib import Path

# logs_path = Path.cwd()/'logs'
# results_path = Path.cwd()/'results'
# logs_path.mkdir(exist_ok=True)
# results_path.mkdir(exist_ok=True)
# log_file_name = logs_path/f'log_out_{datetime.now()}.txt'
# results_file_name = results_path/f'results_out_{datetime.now()}.txt'
# with open(log_file_name, 'w') as file_log:
#     file_log.write('File output\n\n')
# with open(results_file_name, 'w') as file_result:
#     file_result.write('configuration_number,vectorizer_param,model_param,f1\n')

# df_final = df
# df_final['text_final'] = df_final['text_noPunct'].apply(lambda x : ' '.join(x))

# mask_train_test = df_final['sentiment'].notna()

# X_train_valid = df_final.loc[mask_train_test,:]['text_final'].values
# y_train_valid = df_final.loc[mask_train_test,:]['sentiment'].values
# X_test = df_final.loc[~mask_train_test,:]['text_final'].values

# X_train, X_valid, y_train, y_valid = train_test_split(
#     X_train_valid, 
#     y_train_valid, 
#     shuffle=True, 
#     # stratify=y_train_valid, 
#     train_size=0.8, 
#     random_state=50
# )

# configuration_number = 0
# print(f'Number of combinations:\t{len(list(ParameterGrid(TfidfVectorizer_params)))*len(list(ParameterGrid(linearSVC_params)))}')
# for vectorizer_param in ParameterGrid(TfidfVectorizer_params):
#     for model_param in ParameterGrid(linearSVC_params):
#         configuration_number += 1
#         print(configuration_number, vectorizer_param, model_param)

#         # pipe = Pipeline([
#         #     ('tfidf', TfidfVectorizer(**vectorizer_param)),
#         #     ('linearSVC', LinearSVC(**model_param))
#         # ])

#         tfidf = TfidfVectorizer(**vectorizer_param)
#         linearSVC = LinearSVC(**model_param)


#         cuml_X_train = cudf.DataFrame(cudf.Series(X_train, name='text'))
#         cuml_X_valid = cudf.DataFrame(cudf.Series(X_valid, name='text'))
#         cuml_y_train = cupy.asarray(y_train)

#         X_train = tfidf.fit_transform(X_train)
#         X_valid = tfidf.transform(X_valid)

#         linearSVC.fit(X_train,y_train)

#         y_pred = linearSVC.predict(X_valid)

#         y_pred = cupy.asnumpy(y_pred)

#         f1 = f1_score(y_valid, y_pred,average='macro')
#         report = classification_report(y_valid, y_pred)
#         # confusion = confusion_matrix(y_valid, y_pred)

#         with open(log_file_name, 'a') as file_log:
#             file_log.write(f'configuration_number: {configuration_number}\n')
#             file_log.write(f'vectorizer_param: \t{vectorizer_param}\n')
#             file_log.write(f'model_param: \t\t{model_param}\n')
#             file_log.write(f'f1_score:\t\t\t{f1}\n\n')
#             file_log.write(f'{report}\n\n')
#             file_log.write(f'{"*"*150}\n\n')
#         with open(results_file_name, 'a') as file_result:
#             file_result.write(f'{configuration_number},{vectorizer_param},{model_param},{f1}\n')
